In [1]:
import pandas as pd 
import math
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import pickle
style.use('ggplot')

E:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df = pd.read_csv("C:/Users/Dell/Desktop/Python/new_bitcoin_price_Training.csv")
# df.set_index('day',inplace=True)
df.tail()

,Date,Open,High,Low,Close,Volume,Market Cap
1308,31-Dec-13,760.32,760.58,738.17,754.01,"20,897,300.0","9,268,240,000"
1309,30-Dec-13,741.35,766.60,740.24,756.13,"20,707,700.0","9,037,000,000"
1310,29-Dec-13,728.05,748.61,714.44,745.05,"19,011,300.0","8,872,600,000"
1311,28-Dec-13,737.98,747.06,705.35,727.83,"32,505,800.0","8,990,850,000"
1312,27-Dec-13,763.28,777.51,713.60,735.07,"46,862,700.0","9,295,570,000"


In [3]:
type(df['Volume'][0])

str

In [4]:
df['HL_PCT'] = (df['High'] - df['Low'])/ df['Low'] * 100
df['PCT_change'] = (df['Close'] - df['Open'])/ df['Open'] * 100
df = df[['Close','HL_PCT','PCT_change','Volume']]
df.head()

,Close,HL_PCT,PCT_change,Volume
0,2875.34,6.212210,4.056832,"860,575,000.0"
1,2757.18,4.298164,1.203572,705943000.0
2,2726.45,4.306298,-2.870304,"803,746,000.0"
3,2809.01,8.124699,4.824367,"1,380,100,000.0"
4,2671.78,6.483114,5.241638,"789,104,000.0"


In [5]:
forecast = 'Close'
df.fillna(-99999,inplace=True)
forecast_out = int(math.ceil(0.1*len(df)))
df['label'] = df[forecast].shift(-forecast_out)
print(df.head(),'\n',forecast_out)
df.tail()

     Close    HL_PCT  PCT_change           Volume    label
0  2875.34  6.212210    4.056832    860,575,000.0  1120.54
1  2757.18  4.298164    1.203572      705943000.0  1054.23
2  2726.45  4.306298   -2.870304    803,746,000.0  1036.74
3  2809.01  8.124699    4.824367  1,380,100,000.0   973.82
4  2671.78  6.483114    5.241638    789,104,000.0  1100.23 
 132


,Close,HL_PCT,PCT_change,Volume,label
1308,754.01,3.035886,-0.829914,"20,897,300.0",NaN
1309,756.13,3.561007,1.993660,"20,707,700.0",NaN
1310,745.05,4.782767,2.335004,"19,011,300.0",NaN
1311,727.83,5.913376,-1.375376,"32,505,800.0",NaN
1312,735.07,8.955998,-3.695891,"46,862,700.0",NaN


In [6]:
X = np.array(df[['Close','HL_PCT','PCT_change']])
X = preprocessing.scale(X)

X_test = X[-forecast_out:]
X = X[:-forecast_out]

df.dropna(inplace=True)
y = np.array(df['label'])

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X,y,test_size=0.2)

clf = LinearRegression()
clf.fit(X_train,y_train)
accuracy = clf.score(X_test,y_test)
predicted = clf.predict(X_test)
print(predicted, accuracy) #since the input data we got in decreasing price value

[ 468.17797193  390.66330543  367.95274572  424.86029332  385.65720422
  359.03847888  401.73282975  400.23112708  471.44344621  386.28623126
  478.39635559  476.25860693  609.22705167  553.82147161  358.01675265
  361.57874056  532.58634737  359.75845565  527.00786261  403.17408675
  578.63751104  478.21488229  659.90955462  472.77270214  486.14632215
  365.25033556  369.45025072  478.44820884  379.90927958 1244.99109233
  717.70816549  538.17454771  469.48036284  418.98466309  500.45629941
  548.85243559  402.06939849  380.11561463  615.85052777  448.63716786
  406.70842841  482.71433749 1071.42922085  433.77720793  428.32278391
  532.77234016  425.38858598 1128.74575432  492.34859448  662.41330425
  413.01896224  469.09170405  354.41681171  363.51514855  378.48098083
  519.86918954  369.16568003  436.55816484  364.53414423  373.50763799
  584.76849615  436.30620569  416.72960854  371.23193564  361.47367107
  425.78077943  523.63766874  427.37466802  510.27967877 1199.63135805
  372.